In [1]:
#ln44 윗부분은 전처리입니다(굳이 안읽으셔도됩니다.)

In [2]:
#코드 주석 필독 부탁드립니다..

In [1]:
import numpy as np
import pandas as pd


In [5]:
data=pd.read_csv("for_1days_chargedf.csv",encoding='euc-kr')

In [6]:
apart=pd.read_csv("resident_apart.csv",encoding='euc-kr');apart

,Unnamed: 0,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,address,apart_number,adng_nm,cell_id
0,0,5,7,1,6,5,EV라운지,3.522,127.152250,37.286300,경기도 용인시 기흥구 동백8로 9,436,동백1동,86135034
1,1,25,6,1,8,2,파워큐브차저,4.231,127.090080,37.309536,경기도 용인시 수지구 만현로67번길 19,213,상현2동,84375196
2,2,19,7,1,13,2,EV라운지,4.335,127.080970,37.323480,경기도 용인시 수지구 신봉1로48번길 32,151,신봉동,84119243
3,3,10,5,0,13,3,EV라운지,8.648,127.230800,37.246840,경기도 용인시 처인구 한터로152번길 45,566,유림동,88358807
4,4,20,1,1,3,3,EV라운지,12.090,127.071450,37.310825,경기도 용인시 수지구 성복1로 35,3,성복동,83847232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159080,159080,26,7,1,14,5,파워큐브차저,8.530,127.195300,37.242702,경기도 용인시 처인구 낙은로 59,63,역삼동,87350861
159081,159081,20,1,1,13,4,EV라운지,3.912,127.109030,37.244700,경기도 용인시 기흥구 한보라2로 168,264,기흥동,84903019
159082,159082,5,7,1,2,3,EV라운지,5.082,127.079384,37.304993,경기도 용인시 수지구 상현로 59,183,상현1동,84071205
159083,159083,11,6,1,13,2,EV라운지,6.452,127.170300,37.156097,경기도 용인시 처인구 남사읍 한숲로 83,39,남사읍,86630714


In [7]:
import requests, json, pprint

def get_address_info(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json?x="+str(lng)+"&y="+str(lat)
    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
    headers = {"Authorization": "KakaoAK 5cc785c3b89b4883ef1f77d1cc17b935"}
    api_json = requests.get(url, headers=headers)
    address_info = json.loads(api_json.text)
    if address_info['meta']['total_count'] == 0:
        address_info = None

    return address_info

def get_road_address(cell_ycrd, cell_xcrd):
    address_info = get_address_info(cell_ycrd, cell_xcrd)
    
    if address_info == None:
        address_type = None
        full_address = None
    else:
        if address_info['documents'][0]['road_address'] == None:
            address_type = 0
            full_address = address_info['documents'][0]['address']['address_name']
        elif address_info['documents'][0]['road_address'] != None:
            address_type = 1
            full_address = address_info['documents'][0]['road_address']['address_name']
        else:
            address_type = None
            full_adress = None
            
    return full_address

In [8]:
get_road_address(37,127)

'경기 평택시 오성면 교포리 199-4'

In [9]:
data['cell_xcrd'][0]

127.1193559

In [10]:
'경기도 용인시 기흥구 동백8로 9' in list(apart['address'])

True

In [11]:
apartcheck=[]
for i in range(len(data)):
    if get_road_address(data['cell_ycrd'][i],data['cell_xcrd'][i]) in list(apart['address']):
        apartcheck.append(False)
    else:
        apartcheck.append(True)

In [12]:
sum(apartcheck)

209

In [13]:
data[apartcheck]

,Unnamed: 0.1,Unnamed: 0,주차장명,도로명주소,완속충전기수,운영시간,"요금(유료=1, 무료=0)",전체충전기수,1시(이용량),3시,...,9시,11시,13시,15시,17시,19시,21시,23시,cell_xcrd,cell_ycrd
0,0,0,기흥ICT밸리 A동 전기차충전소,경기 용인시 기흥구 기흥로 58-1,12,9,1,12,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,127.119356,37.280023
1,3,3,이케아 기흥점 전기차충전소,경기 용인시 기흥구 신고매로 62 P1주차장,7,24,1,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.116183,37.222626
2,4,4,모현면사무소 전기차충전소,경기 용인시 처인구 모현읍 독점로 31-6,0,24,1,1,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,127.242362,37.329533
3,5,5,고영테크놀러지 RnD센터 전기차충전소,경기 용인시 수지구 포은대로59번길 18,10,9,1,13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.066816,37.301184
4,6,6,용인시 여성회관 주차장 전기차충전소,경기 용인시 수지구 문정로7번길 7 지하1층,0,24,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,127.096154,37.321033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,480,480,동백중앙프라자상가 전기차충전소,경기 용인시 기흥구 동백7로 83,5,24,1,5,2.5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,127.160396,37.281685
458,484,484,경기도 용인시 기흥구 동백중앙로54번길 6-37,경기 용인시 기흥구 동백중앙로54번길 6-37,0,9,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.163192,37.261510
460,486,486,기흥SK아펠바움콘도미니엄 전기차충전소,경기 용인시 기흥구 기흥단지로 394,3,24,1,3,1.5,1.5,...,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,127.133125,37.217566
463,489,489,쥬네브문월드 전기차충전소,경기 용인시 기흥구 동백5로 22,6,24,1,6,0.0,0.0,...,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,127.151277,37.275857


In [14]:
apcheck=data[apartcheck]

In [15]:
list(apcheck['주차장명'])[0]

'기흥ICT밸리 A동 전기차충전소'

In [16]:
secondout=[]
for i in range(len(apcheck)):
    if "아파트" in list(apcheck['주차장명'])[i]:
        secondout.append(False)
    else:
        secondout.append(True)
        

In [17]:
sum(secondout)

199

In [18]:
newapcheck=apcheck[secondout]

In [ ]:
#newapcheck.to_csv("nonapart 1day charge.csv",encoding='euc-kr')

In [ ]:
#여기 윗부분까지는 아파트 주차장이 아닌 주차장크롤링 데이터를 뽑아오는 과정입니다.
#용인시 아파트 데이터와 도로명주소가 일치하거나 주차장명에 아파트가 들어가는 행을 삭제했습니다.

In [ ]:
#이렇게 모아진 newapcheck데이터에서 보니 한가지 문제가 완속충전소 수> 전체 충전소 수 인 행이 4개 있었습니다.
#다른 행들은 다 맞게 크롤링된것 같은데 이유를 모르겠습니다. 카카오맵에 수동으로 접속해서 수정했습니다,
#null 값 ->0으로 치환했습니다.

In [19]:
newdata=pd.read_csv("nonapart 1day chargerev.csv",encoding='euc-kr');newdata

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,주차장명,도로명주소,완속충전기수,운영시간,"요금(유료=1, 무료=0)",전체충전기수,1시(이용량),...,9시,11시,13시,15시,17시,19시,21시,23시,cell_xcrd,cell_ycrd
0,0,0,0,기흥ICT밸리 A동 전기차충전소,경기 용인시 기흥구 기흥로 58-1,12,9,1,12,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,127.119356,37.280023
1,1,3,3,이케아 기흥점 전기차충전소,경기 용인시 기흥구 신고매로 62 P1주차장,7,24,1,8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.116183,37.222626
2,2,4,4,모현면사무소 전기차충전소,경기 용인시 처인구 모현읍 독점로 31-6,0,24,1,1,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,127.242362,37.329533
3,3,5,5,고영테크놀러지 RnD센터 전기차충전소,경기 용인시 수지구 포은대로59번길 18,10,9,1,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.066816,37.301184
4,4,6,6,용인시 여성회관 주차장 전기차충전소,경기 용인시 수지구 문정로7번길 7 지하1층,0,24,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,127.096154,37.321033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,454,480,480,동백중앙프라자상가 전기차충전소,경기 용인시 기흥구 동백7로 83,5,24,1,5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,127.160396,37.281685
195,458,484,484,경기도 용인시 기흥구 동백중앙로54번길 6-37,경기 용인시 기흥구 동백중앙로54번길 6-37,0,9,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.163192,37.261510
196,460,486,486,기흥SK아펠바움콘도미니엄 전기차충전소,경기 용인시 기흥구 기흥단지로 394,3,24,1,3,1.5,...,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,127.133125,37.217566
197,463,489,489,쥬네브문월드 전기차충전소,경기 용인시 기흥구 동백5로 22,6,24,1,6,0.0,...,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,127.151277,37.275857


In [20]:
newdata.iloc[:,9]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
194    2.5
195    0.0
196    1.5
197    0.0
198    4.0
Name: 1시(이용량), Length: 199, dtype: float64

In [21]:
usagelst=[]
for i in range(len(newdata)):
    usage=0
    for j in range(9,21):
        usage+=newdata.iloc[:,j][i]
    usagelst.append(usage/12)

In [22]:
len(usagelst)

199

In [23]:
timeratio=list(newdata['운영시간'])

In [24]:
timeratio=[i/24 for i in timeratio];timeratio

[0.375,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4583333333333333,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4166666666666667,
 0.375,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 0.375,
 0.375,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 0.7916666666666666,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 0.625,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.375,
 0.375,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 0.375,
 1.0,
 0.5416666666666666,
 1.0,
 1.0,
 0.375,
 1.0,
 1.0,
 0.5,
 0.375,
 1.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 0.375,
 0.375,
 0.375,
 1.0,
 0.375,
 1.0,


In [25]:
slowchargenumlst=list(newdata['완속충전기수'])

In [26]:
totalchargenumlst=list(newdata['전체충전기수'])

In [27]:
fastchargenumlst=[]
for i in range(len(slowchargenumlst)):
    a=totalchargenumlst[i]-slowchargenumlst[i]
    fastchargenumlst.append(a)

In [28]:
fastchargenumlst

[0,
 1,
 1,
 3,
 1,
 8,
 0,
 3,
 4,
 1,
 1,
 0,
 1,
 0,
 0,
 3,
 12,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 5,
 2,
 2,
 1,
 4,
 12,
 0,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 0,
 1,
 0,
 0,
 4,
 2,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 5,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 1,
 0,
 1,
 4,
 7,
 1,
 1,
 0,
 1,
 0,
 3,
 0,
 1,
 1,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 9,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 4,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

In [ ]:
#급속충전이 완속충전보다 4배 빠르다고 계산, 4배의 가중치(풀충전까지 완속 4시간, 급속 1시간)

In [ ]:
#추정된 충전대수는  평균이용량(%)*시간비(24시간 기준 1)*(완속충전기수+4*급속충전기수) 에 비례한다.

In [29]:
totalestimate=[]
for i in range(len(usagelst)):
    if totalchargenumlst[i]!=0:
        avgratio=usagelst[i]/totalchargenumlst[i]
    else:
        avgratio=0
    tot=avgratio*timeratio[i]*((slowchargenumlst[i])+4*(fastchargenumlst[i]))
    totalestimate.append(tot)
    if totalestimate[i]<0:
        print(i)

In [30]:
totalestimate

[0.0625,
 0.0,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 2.6666666666666665,
 0.75,
 0.0,
 6.666666666666667,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0625,
 3.453125,
 24.0,
 4.0,
 0.0,
 0.5,
 0.16666666666666666,
 3.0,
 0.6666666666666666,
 0.0,
 0.8846153846153846,
 0.3333333333333333,
 4.0,
 0.8333333333333335,
 7.666666666666667,
 4.0,
 0.0,
 5.0,
 1.6666666666666667,
 0.0,
 2.0,
 1.0,
 2.6666666666666665,
 2.0,
 0.6666666666666666,
 2.0,
 1.6666666666666667,
 4.0,
 1.1666666666666667,
 2.0,
 1.3333333333333333,
 1.1458333333333333,
 4.0,
 2.0,
 0.3333333333333333,
 0.0,
 2.0,
 1.6666666666666667,
 2.0,
 0.125,
 0.25,
 0.0,
 0.3333333333333333,
 1.0,
 0.08333333333333333,
 2.0,
 0.16666666666666666,
 4.0,
 0.5555555555555556,
 0.15625,
 0.0,
 0.982142857142857,
 0.0,
 0.0,
 0.6666666666666666,
 1.3333333333333333,
 0.16666666666666666,
 2.0,
 0.0,
 0.15625,
 0.25,
 0.75,
 2.0,
 0.75,
 0.0,
 1.0,
 0.0,
 0.3333333333333333,
 5.0,
 4.0,
 0.5,
 0.0,
 2.3333333333333335,
 1.986111111523

In [31]:
total=0
for i in totalestimate:
    total+=i
print(total)

255.89111340090116


In [ ]:
#크롤링 당시 고려되지 않았지만 잠재적으로 운영중이지 않는 충전소 역시 이용될 수 있다.
#하지만 접근성 문제 및 충전 과정에서의 불편함(충전 시설 자체의 문제) 등으로 수요가 있어도 잘 활용되지 못하고 있다
#그래서 여분의 충전 가능 시설에 대해서는 0.3의 가중치를 주어 잠재적으로(maximal coverage가 될경우 상한) 수용된다고 가정한다.

In [32]:
potentialusagelst=[]
for i in range(len(usagelst)):
    if totalchargenumlst[i]!=0:
        avgratio=usagelst[i]/totalchargenumlst[i]
    else:
        avgratio=0
    potentialusagelst.append(1-avgratio)

In [33]:
potentialusagelst

[0.9861111111111112,
 1.0,
 0.8333333333333334,
 1.0,
 0.9166666666666666,
 0.9166666666666666,
 0.75,
 1.0,
 0.5833333333333333,
 1.0,
 0.75,
 1.0,
 1.0,
 0.5,
 0.9166666666666666,
 0.45833333333333337,
 0.5,
 0.5,
 1.0,
 0.875,
 0.9444444444444444,
 0.25,
 0.8333333333333334,
 1.0,
 0.8525641025641025,
 0.9166666666666666,
 0.5,
 0.9583333333333334,
 0.04166666666666663,
 0.5,
 1.0,
 0.6875,
 0.9305555555555556,
 1.0,
 0.5,
 0.75,
 0.8333333333333334,
 0.5,
 0.8333333333333334,
 0.5,
 0.5833333333333333,
 0.5,
 0.9027777777777778,
 0.5,
 0.6666666666666667,
 0.7708333333333334,
 0.5,
 0.5,
 0.8333333333333334,
 1.0,
 0.5,
 0.5833333333333333,
 0.5,
 0.9166666666666666,
 0.8333333333333334,
 1.0,
 0.9166666666666666,
 0.9375,
 0.9583333333333334,
 0.5,
 0.9166666666666666,
 0.5,
 0.9166666666666666,
 0.9722222222222222,
 1.0,
 0.8908730158730159,
 1.0,
 1.0,
 0.8333333333333334,
 0.5555555555555556,
 0.9166666666666666,
 0.75,
 1.0,
 0.8958333333333334,
 0.8333333333333334,
 0.625,
 0

In [ ]:
#그러면 현재 용인시에 있는 non-apartment 충전소의 coverage는 아래의 final_estimate값에 비례하게 된다.

In [34]:
final_estimate=[]
for i in range(len(totalestimate)):
    a=totalestimate[i]+0.3*potentialusagelst[i]*totalchargenumlst[i]
    final_estimate.append(a)

In [35]:
min(final_estimate)

0.3

In [36]:
max(final_estimate)

25.8

In [37]:
import numpy as np

In [38]:
np.sum(final_estimate)

394.1202800675011

In [ ]:
#전국 전기차 충전장소 1기당 coverage는 평균 2.6대이다(22년 10월)
#출처 : https://eiec.kdi.re.kr/userdata/nara/202210/202210.pdf

In [ ]:
#전기차 충전소 1기당 optimal coverage는 3대라고 가정하겠다.(OECD에서 매우 높은 비율이라고 하기도 하였고, 
#최적입지로 접근성을 극대화, 유지보수가 잘 이루어진다면 문제없이 운영될 수 있다. )


In [39]:
nonapart=pd.read_csv("resident_notapart.csv",encoding='euc-kr');nonapart

,Unnamed: 0,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,address,apart_number,adng_nm,cell_id
0,0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,경기 용인시 처인구 포곡읍 둔전리 42-2,0,포곡읍,87958893
1,1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,경기 용인시 수지구 상현동 산 13-6,0,상현2동,84423187
2,2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,경기도 용인시 기흥구 상갈로29번길 10,0,상갈동,84855076
3,3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,경기 용인시 수지구 풍덕천동 764,0,풍덕천1동,84695224
4,4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,경기도 용인시 처인구 금령로40번길 8,0,중앙동,87494833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132953,132953,6,1,0,10,2,EV라운지,3.629,127.194190,37.236390,경기도 용인시 처인구 중부대로1343번길 11-3,0,역삼동,87318849
132954,132954,17,5,1,3,3,EV라운지,4.463,127.238070,37.263084,경기 용인시 처인구 포곡읍 금어리 527,0,포곡읍,88566830
132955,132955,10,5,0,10,5,EV라운지,3.227,127.101160,37.241520,경기 용인시 기흥구 공세동 476-343,0,기흥동,84679026
132956,132956,7,2,1,3,6,EV라운지,4.585,127.193080,37.234580,경기 용인시 처인구 김량장동 508-1,0,중앙동,87286847


In [ ]:
#전국의 충전장소 1기당 평균 충전하는 차량 수 자료를 이용하여 용인시에 적용시킨다.

In [40]:
sum(totalchargenumlst)

579

In [41]:
579*3

1737

In [ ]:
#그러므로 용인시에 있는 비아파트 충전소 127곳, 579기의 충전장소는 이상적으로 
#약 1737대를 커버하기위해 설립되었다.

In [42]:
weights=1737/np.sum(final_estimate);weights

4.407283988792719

In [ ]:
#위의 식에 따라 비례식(final_estimate)에 weight를 곱해주어, 1기당 평균 3대를 커버할 수 있도록 조정해 준다.

In [43]:
final_weighted_estimate=[weights*i for i in final_estimate]

In [44]:
final_weighted_estimate

[15.921313409513697,
 10.577481573102526,
 4.040010323059993,
 17.188407556291605,
 2.681097759848904,
 21.448782078791233,
 6.280379684029624,
 3.966555589913447,
 32.46699205077304,
 1.3221851966378158,
 5.398922886271081,
 7.933111179826894,
 1.3221851966378158,
 5.729469185430536,
 2.69946144313554,
 20.066914911471848,
 113.70792691085217,
 18.951321151808692,
 2.6443703932756315,
 6.831290182628715,
 4.4807387219392645,
 13.552398265537613,
 4.040010323059993,
 1.3221851966378158,
 18.552970483590887,
 2.681097759848904,
 18.951321151808692,
 10.0082073912168,
 33.89935934713067,
 18.951321151808692,
 1.3221851966378158,
 25.67242923471759,
 22.10987467711014,
 2.6443703932756315,
 9.475660575904346,
 5.398922886271081,
 16.16004129223997,
 9.475660575904346,
 4.040010323059993,
 9.475660575904346,
 8.11674801269326,
 18.951321151808692,
 12.303667802046343,
 9.475660575904346,
 6.757835449482169,
 10.145935015866572,
 18.951321151808692,
 9.475660575904346,
 3.672736657327266,
 

In [45]:
sum(final_weighted_estimate)

1737.0

In [46]:
#결과적으로, 이용빈도가 높거나, 급속충전기의 비율이 높거나, 전체 충전기의 수가 많을수록,
#주차장의 maximal coverage가 높음을 확인할 수 있다.

In [47]:
#한가지 문제는, 위의 final_weighted_estimate가 이상적인 coverage라는 것이다.
#실제로는 충전소의 접근성과 충전소 시설 문제로 인해서 1기당 부담하는 커버량이 훨씬 클 것이다.


In [48]:
#출처: https://www.data.go.kr/data/15106940/fileData.do
real_elec_car=5957    #용인시 전기차 등록 대수(22년 10월 기준)

In [49]:
#이중, 아파트 충전소가 아닌 다른 충전소를 쓰는 차량의 수를 추산해 보자.
r=sum(nonapart['count_cust'])/sum(apart['count_cust'])
print(r)
err=10000
#총 5957대를 아파트 비아파트로 쪼개서 ratio가 0.81이 나오도록 x를 찾는 식입니다.(2차방정식 풀이)
for x in range(2000,4000):
    if ((x/(real_elec_car-x))-r)**2<err:
        err=((x/(real_elec_car-x))-r)**2
        ans=x
print(ans)

0.8119983094272534
2669


In [50]:
#위의 자료에 의하면 현재의 coverage 상황은 훨씬 열악하다.
#주어진 199곳, 579기의 충전소가 2669대의 수요를 해결하고있다고 가정했을때, 1기당 담당하는 차량 대수.
add_weight=ans/sum(final_weighted_estimate);
real_weight=add_weight*3;real_weight

4.609671848013817

In [51]:
#그래서 전기차 충전소의 추가적인 설립이 필요한 이유를 다시한번 확인할 수 있다,
#향후의 목표 : 기당 4.6대를 커버하는 현실!
#목표하는 상황을 가정했을 때(기당 2대를 커버함) 수요점을 제외한 후, 나머지 수요점에 대해 추가적인 충전소를 설치한다!


In [52]:
have_to_cover=2669-1158;have_to_cover

1511

In [53]:
#출처 https://news.kbs.co.kr/news/view.do?ncd=5575770
#급속충전기 vs 완속충전기 비율 = 1:6

In [54]:
#장소당 10개의 충전기를 설치한다면, 현재로서는 약 150군데(급속21, 완속 129)에 충전소를 설립해야한다.
#전부 완속으로 할경우, 급속의 효율이 완속의 4배 이므로(충전 시간기준)->약 200군데 필요할 것으로 추산
#https://www.korea.kr/news/policyNewsView.do?newsId=148890896
#위의 출처를 봤을 떄, 향후 몇년을 고려하는가에 따라 추가적으로 가정을 해야할 것이다.
#예를 들어, 급속을 늘린다. 혹은 충전소와 충전기를 늘린다 등 etc.

In [55]:
#마지막으로 고려해야 할점
#number of coverage 와 앱 실행 빈도는 비례할 뿐 동일관계가 아니다.

In [56]:
totcountcust_inmonth=sum(nonapart['count_cust']);totcountcust_inmonth

859917.5329999744

In [57]:
coveragecountcust=totcountcust_inmonth*3/4.6;coveragecountcust

560815.7823912876

In [58]:
#우리는 현재 (3/4.6=65%)에 해당되는 수요점만 커버할 수 있기 떄문에 총 65%에 대한 수요점은 지워야한다.
#하지만 격자 내 총 수요보다 충전소의 coveragecountcust가 크다면 수요를 음수로 만들수는 없으므로 
#65%보다 조금 적은 수요점이 지워질 수도 있다.

In [59]:
countcust_weight=coveragecountcust/sum(final_weighted_estimate);countcust_weight

322.8645839903786

In [60]:
countcust_weighted_estimate=[countcust_weight*i for i in final_weighted_estimate]

In [61]:
newdata['car_coverage']=final_weighted_estimate
newdata['countcust_coverage']=countcust_weighted_estimate

In [62]:
newdata

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,주차장명,도로명주소,완속충전기수,운영시간,"요금(유료=1, 무료=0)",전체충전기수,1시(이용량),...,13시,15시,17시,19시,21시,23시,cell_xcrd,cell_ycrd,car_coverage,countcust_coverage
0,0,0,0,기흥ICT밸리 A동 전기차충전소,경기 용인시 기흥구 기흥로 58-1,12,9,1,12,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,127.119356,37.280023,15.921313,5140.428231
1,1,3,3,이케아 기흥점 전기차충전소,경기 용인시 기흥구 신고매로 62 P1주차장,7,24,1,8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,127.116183,37.222626,10.577482,3415.094188
2,2,4,4,모현면사무소 전기차충전소,경기 용인시 처인구 모현읍 독점로 31-6,0,24,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,127.242362,37.329533,4.040010,1304.376252
3,3,5,5,고영테크놀러지 RnD센터 전기차충전소,경기 용인시 수지구 포은대로59번길 18,10,9,1,13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,127.066816,37.301184,17.188408,5549.528055
4,4,6,6,용인시 여성회관 주차장 전기차충전소,경기 용인시 수지구 문정로7번길 7 지하1층,0,24,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,127.096154,37.321033,2.681098,865.631513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,454,480,480,동백중앙프라자상가 전기차충전소,경기 용인시 기흥구 동백7로 83,5,24,1,5,2.5,...,2.5,2.5,2.5,2.5,2.5,2.5,127.160396,37.281685,14.323673,4624.606713
195,458,484,484,경기도 용인시 기흥구 동백중앙로54번길 6-37,경기 용인시 기흥구 동백중앙로54번길 6-37,0,9,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,127.163192,37.261510,1.322185,426.886773
196,460,486,486,기흥SK아펠바움콘도미니엄 전기차충전소,경기 용인시 기흥구 기흥단지로 394,3,24,1,3,1.5,...,1.5,1.5,1.5,1.5,1.5,1.5,127.133125,37.217566,8.594204,2774.764028
197,463,489,489,쥬네브문월드 전기차충전소,경기 용인시 기흥구 동백5로 22,6,24,1,6,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,127.151277,37.275857,8.318749,2685.829283


In [63]:
#newdata.to_csv("notapart_charge_coverage.csv",encoding='euc-kr')

##개인적으로 고민한 내용입니다(ignore from here)##

In [64]:
import pandas as pd

In [65]:
outactivity=pd.read_csv("3.ev_app_activity_resident.csv",encoding='euc-kr');outactivity

,base_dt,dow,mega_cd,mega_nm,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220611,6,41,경기도,4146,용인시,41461253,모현읍,86695054,127.171880,37.311146,FEMALE,13,EV라운지,1,4.492
1,20220605,7,11,서울특별시,1168,강남구,11680580,삼성1동,83591688,127.061264,37.509087,MALE,6,EV라운지,4,3.666
2,20220602,4,11,서울특별시,1120,성동구,11200670,성수2가1동,83463765,127.056560,37.540170,MALE,10,EV라운지,4,3.247
3,20220608,3,41,경기도,4146,용인시,41463510,신갈동,84951117,127.110490,37.290226,FEMALE,13,EV라운지,3,6.664
4,20220608,3,41,경기도,4150,이천시,41500330,호법면,92774456,127.386450,37.213290,MALE,11,EV라운지,5,3.945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293166,20220609,4,41,경기도,4146,용인시,41465530,신봉동,83943267,127.074730,37.329315,FEMALE,11,EV라운지,3,3.396
293167,20220626,7,41,경기도,4146,용인시,41465540,죽전1동,85719164,127.137360,37.333126,MALE,8,EV라운지,6,4.320
293168,20220607,2,41,경기도,4113,성남시,41131625,위례동,85863462,127.141785,37.471504,MALE,10,EV라운지,4,11.457
293169,20220619,7,11,서울특별시,1168,강남구,11680640,역삼1동,82791718,127.033035,37.499966,MALE,8,하이차저,5,4.232


In [66]:
outyongin=outactivity[outactivity['ccw_nm']=="용인시"]

In [67]:
sum(outyongin['count_cust'])

1282000.8200000683

In [68]:
activitytwo=pd.read_csv("2.ev_app_activity.csv",encoding='euc-kr')

In [69]:
b=sum(activitytwo['count_cust']);b

1771148.7990000322

In [70]:
activityone=pd.read_csv("1.ev_app_resident.csv",encoding='euc-kr')

In [71]:
a=sum(activityone['count_cust']);a

1918931.4780002383

In [72]:
#주간인구지수(유출이 더 많다.)
b/a

0.9229869952656081

app count기준
3번 데이터 : 용인시 거주자 이면서 용인 시내 활동자 약 128만 건
2번 데이터 : 용인시 총 활동인구 약 177만 건
1번 데이터 : 용인시 총 거주자 약 191만 건

데이터를 통해 알수 있는 것: 유출 활동자는 63만건 유입 활동자는 49만건
포함관계 : 3이 1번, 2번의 교집합
1번/2번(약 0.92, 용인시의 경우 유출인구가 더 많다는 결론, 용인시 주간인구지수 통계자료랑 일치함)

전체의 (128/191*100) %의 차량은 유지, 전체 (63/191*100)%만큼 유출, 전체 (49/191*100)%만큼 유입

In [73]:
#여기서 부터 다시 시작합니다.

In [4]:
activitytwo=pd.read_csv("G:/내 드라이브/ESC-22FALL/빅콘테스트/data/2.ev_app_activity.csv",encoding='utf-8');activitytwo

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220614,2,4146,용인시,41461256,이동읍,87958702,127.216920,37.188220,MALE,12,EV라운지,4,3.829
1,20220613,1,4146,용인시,41463570,마북동,85431116,127.127350,37.303352,MALE,14,EV라운지,1,9.994
2,20220602,4,4146,용인시,41463575,동백2동,86198990,127.154590,37.268280,MALE,14,EV라운지,4,7.502
3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.178110,37.174600,MALE,9,EV라운지,4,3.234
4,20220608,3,4146,용인시,41465540,죽전1동,85159183,127.117645,37.325855,FEMALE,10,EV라운지,5,3.315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264708,20220621,2,4146,용인시,41461253,모현읍,86023090,127.148200,37.308372,MALE,7,EV라운지,2,7.140
264709,20220628,2,4146,용인시,41465530,신봉동,84103247,127.080400,37.324825,MALE,3,EV라운지,5,21.564
264710,20220626,7,4146,용인시,41463510,신갈동,84375154,127.090180,37.290607,MALE,12,EV라운지,4,3.502
264711,20220620,1,4146,용인시,41465590,성복동,83895245,127.073105,37.318040,MALE,14,EV라운지,3,11.406


In [75]:
#유입인구를 추산할 방법이 시간적으로 부족했습니다. 비율대로 random하게 가져와,유입인구의 수요라고 가정합니다.

In [5]:
acttwo=activitytwo.sample(frac=49/177, replace=False)

In [6]:
acttwo

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
252267,20220615,3,4146,용인시,41463550,서농동,83927058,127.07470,37.234670,MALE,10,EV라운지,2,3.953
157074,20220625,6,4146,용인시,41465530,신봉동,84151231,127.08212,37.318974,MALE,9,EV라운지,5,4.320
173317,20220630,4,4146,용인시,41463530,상갈동,84679080,127.10103,37.265858,MALE,8,EV라운지,6,8.346
180231,20220615,3,4146,용인시,41465580,상현2동,84439198,127.09232,37.312250,FEMALE,14,EV라운지,4,4.519
230675,20220601,3,4146,용인시,41463570,마북동,85447125,127.12789,37.307860,FEMALE,10,EV라운지,2,4.254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205535,20220623,4,4146,용인시,41463590,보정동,84631160,127.09915,37.300556,MALE,14,EV라운지,4,8.732
257751,20220628,2,4146,용인시,41465520,풍덕천2동,84295216,127.08722,37.316288,FEMALE,6,하이차저,4,3.973
253872,20220605,7,4146,용인시,41461520,역삼동,87094858,127.18631,37.234116,FEMALE,14,EV라운지,4,18.080
53228,20220616,4,4146,용인시,41463577,동백3동,86007038,127.14774,37.284485,MALE,14,EV라운지,5,8.418


In [7]:
acttwo["base_dt"]=[str(i)[-2:] for i in list(acttwo['base_dt'])]

In [8]:
acttwo=acttwo.rename(columns={'base_dt':'date'})

In [10]:
base=pd.read_csv("G:/내 드라이브/ESC-22FALL/빅콘테스트/preprocessed data/resident_notapart.csv",encoding='euc-kr');base

,Unnamed: 0,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,address,apart_number,adng_nm,cell_id
0,0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,경기 용인시 처인구 포곡읍 둔전리 42-2,0,포곡읍,87958893
1,1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,경기 용인시 수지구 상현동 산 13-6,0,상현2동,84423187
2,2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,경기도 용인시 기흥구 상갈로29번길 10,0,상갈동,84855076
3,3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,경기 용인시 수지구 풍덕천동 764,0,풍덕천1동,84695224
4,4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,경기도 용인시 처인구 금령로40번길 8,0,중앙동,87494833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132953,132953,6,1,0,10,2,EV라운지,3.629,127.194190,37.236390,경기도 용인시 처인구 중부대로1343번길 11-3,0,역삼동,87318849
132954,132954,17,5,1,3,3,EV라운지,4.463,127.238070,37.263084,경기 용인시 처인구 포곡읍 금어리 527,0,포곡읍,88566830
132955,132955,10,5,0,10,5,EV라운지,3.227,127.101160,37.241520,경기 용인시 기흥구 공세동 476-343,0,기흥동,84679026
132956,132956,7,2,1,3,6,EV라운지,4.585,127.193080,37.234580,경기 용인시 처인구 김량장동 508-1,0,중앙동,87286847


In [11]:
base.iloc[:,1:]

,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,address,apart_number,adng_nm,cell_id
0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,경기 용인시 처인구 포곡읍 둔전리 42-2,0,포곡읍,87958893
1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,경기 용인시 수지구 상현동 산 13-6,0,상현2동,84423187
2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,경기도 용인시 기흥구 상갈로29번길 10,0,상갈동,84855076
3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,경기 용인시 수지구 풍덕천동 764,0,풍덕천1동,84695224
4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,경기도 용인시 처인구 금령로40번길 8,0,중앙동,87494833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132953,6,1,0,10,2,EV라운지,3.629,127.194190,37.236390,경기도 용인시 처인구 중부대로1343번길 11-3,0,역삼동,87318849
132954,17,5,1,3,3,EV라운지,4.463,127.238070,37.263084,경기 용인시 처인구 포곡읍 금어리 527,0,포곡읍,88566830
132955,10,5,0,10,5,EV라운지,3.227,127.101160,37.241520,경기 용인시 기흥구 공세동 476-343,0,기흥동,84679026
132956,7,2,1,3,6,EV라운지,4.585,127.193080,37.234580,경기 용인시 처인구 김량장동 508-1,0,중앙동,87286847


In [14]:
first=acttwo[["date","dow","gender","age","time_zone","app_web","count_cust","cell_xcrd","cell_ycrd","adng_nm","cell_id"]]
first['res'] = 1
first

C:\Users\Yubin Hong\AppData\Local\Temp\ipykernel_13768\867950478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['res'] = 1


,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,adng_nm,cell_id,res
252267,15,3,MALE,10,2,EV라운지,3.953,127.07470,37.234670,서농동,83927058,1
157074,25,6,MALE,9,5,EV라운지,4.320,127.08212,37.318974,신봉동,84151231,1
173317,30,4,MALE,8,6,EV라운지,8.346,127.10103,37.265858,상갈동,84679080,1
180231,15,3,FEMALE,14,4,EV라운지,4.519,127.09232,37.312250,상현2동,84439198,1
230675,01,3,FEMALE,10,2,EV라운지,4.254,127.12789,37.307860,마북동,85447125,1
...,...,...,...,...,...,...,...,...,...,...,...,...
205535,23,4,MALE,14,4,EV라운지,8.732,127.09915,37.300556,보정동,84631160,1
257751,28,2,FEMALE,6,4,하이차저,3.973,127.08722,37.316288,풍덕천2동,84295216,1
253872,05,7,FEMALE,14,4,EV라운지,18.080,127.18631,37.234116,역삼동,87094858,1
53228,16,4,MALE,14,5,EV라운지,8.418,127.14774,37.284485,동백3동,86007038,1


In [16]:
second=base[["date","dow","gender","age","time_zone","app_web","count_cust","cell_xcrd","cell_ycrd","adng_nm","cell_id"]]
second['res'] = 0
second

C:\Users\Yubin Hong\AppData\Local\Temp\ipykernel_13768\3003386246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second['res'] = 0


,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,adng_nm,cell_id,res
0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,포곡읍,87958893,0
1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,상현2동,84423187,0
2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,상갈동,84855076,0
3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,풍덕천1동,84695224,0
4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,중앙동,87494833,0
...,...,...,...,...,...,...,...,...,...,...,...,...
132953,6,1,0,10,2,EV라운지,3.629,127.194190,37.236390,역삼동,87318849,0
132954,17,5,1,3,3,EV라운지,4.463,127.238070,37.263084,포곡읍,88566830,0
132955,10,5,0,10,5,EV라운지,3.227,127.101160,37.241520,기흥동,84679026,0
132956,7,2,1,3,6,EV라운지,4.585,127.193080,37.234580,중앙동,87286847,0


In [84]:
#기존의 수요점 데이터랑 합칩니다.
#기존의 수요점들은 non apart charger를 이용하는 용인시 거주민 + 용인시 미거주자이면서 용인시 유입인구로 구성됩니다.

In [17]:
res=pd.concat([second,first],axis=0)

In [86]:
#전처리가 안된 acttwo랑 합칩니다(gender)...필요한 정보에 대한 전처리는 끝났기 때문에 일단 무시하고 진행합니다.

In [18]:
res

,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,adng_nm,cell_id,res
0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,포곡읍,87958893,0
1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,상현2동,84423187,0
2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,상갈동,84855076,0
3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,풍덕천1동,84695224,0
4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,중앙동,87494833,0
...,...,...,...,...,...,...,...,...,...,...,...,...
205535,23,4,MALE,14,4,EV라운지,8.732,127.099150,37.300556,보정동,84631160,1
257751,28,2,FEMALE,6,4,하이차저,3.973,127.087220,37.316288,풍덕천2동,84295216,1
253872,05,7,FEMALE,14,4,EV라운지,18.080,127.186310,37.234116,역삼동,87094858,1
53228,16,4,MALE,14,5,EV라운지,8.418,127.147740,37.284485,동백3동,86007038,1


In [20]:
res.reset_index(drop=True, inplace=True)
res

,date,dow,gender,age,time_zone,app_web,count_cust,cell_xcrd,cell_ycrd,adng_nm,cell_id,res
0,14,2,0,14,1,EV라운지,18.440,127.216600,37.274303,포곡읍,87958893,0
1,24,5,1,8,4,EV라운지,4.231,127.091780,37.306840,상현2동,84423187,0
2,12,7,1,12,5,EV라운지,7.448,127.107216,37.269030,상갈동,84855076,0
3,20,1,0,12,4,EV라운지,22.800,127.101250,37.331207,풍덕천1동,84695224,0
4,9,4,1,12,3,EV라운지,11.535,127.200400,37.234150,중앙동,87494833,0
...,...,...,...,...,...,...,...,...,...,...,...,...
206235,23,4,MALE,14,4,EV라운지,8.732,127.099150,37.300556,보정동,84631160,1
206236,28,2,FEMALE,6,4,하이차저,3.973,127.087220,37.316288,풍덕천2동,84295216,1
206237,05,7,FEMALE,14,4,EV라운지,18.080,127.186310,37.234116,역삼동,87094858,1
206238,16,4,MALE,14,5,EV라운지,8.418,127.147740,37.284485,동백3동,86007038,1


In [21]:
res.to_csv("non_apart_estimated_countcust.csv",encoding='euc-kr')

In [89]:
#유입인구에 대한 수요점을 추가하였으므로 real_weight가 수정됩니다.
#추정한 customer ratio를 역산해서 커버해야할 전기차 대수를 계산합니다..

In [90]:
sum(acttwo['count_cust'])

491487.65999999543

In [91]:
sum(acttwo['count_cust'])/countcust_weight

1522.2718265520316

In [92]:
#약 1525대의 전기차를 추가적으로 커버해야한다는 결론이 나옵니다.
#매번 랜덤하게 뽑다보니 코드 파일 재점검 과정에서 약간 다른 값이 나오기도 합니다.
#1525대로 생각하고 논의를 진행하겠습니다

In [93]:
# 2669/579로 산출한 real weight은 이제 (2669+1525)/579 로 계산됩니다.

In [94]:
(2669+1525)/579

7.243523316062176

In [95]:
revised_totcountcust_inmonth=totcountcust_inmonth+sum(acttwo['count_cust']);revised_totcountcust_inmonth

1351405.19299997

In [96]:
revised_coveragecountcust=revised_totcountcust_inmonth*3/7.24;coveragecountcust

560815.7823912876

In [97]:
#coverage ratio

In [98]:
3/7.24


0.4143646408839779

In [99]:
#추가 전기차 대수 수요(첫번쨰 샘플 기반 1525)
489798.49899999535/countcust_weight

1517.0400325313828

In [100]:
#총 non apart 전기차 대수 수요
2669+1525

4194

In [101]:
#총 커버해야하는 양(충전소 신규 설치가 필요한 전기차 대수)
num=4194*(1-3/7.24);num

2456.154696132597

In [102]:
#필요한 충전기수
num/3

818.718232044199

In [103]:
#필요한 완속 충전기수(급속 완속 비율 고려)
num/3*10/7

1169.5974743488557

In [104]:
#1년 뒤의 최소 수요증가치까지는 고려한다.(최소 33%의 성장세를 보일 것이라고 가정)

In [105]:
1169*1.33

1554.77